In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Postgres info to connect

connection_args = {
  "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
  "user": "postgres",
    "password":"FwwBFmleh65qYxKxDVb9",
  "port": 5432,
  "dbname": "twitchdata"
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [36]:
query = "SELECT COUNT(DISTINCT(stream_id)) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,156326


In [46]:
query = "SELECT * FROM stream_data LIMIT 3;"

pd_sql.read_sql(query, connection)

,stream_id,user_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged
0,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24
1,34368684464,273702366,Soddrek,19619,live,HOJE TEM GAZ AS 20H (18+) !burro !loots !lutab...,18,2019-06-01 22:48:35,pt,2019-06-01 23:03:24
2,34365706032,117766148,kyllini,19619,live,PxG Emerald - Kyllini - #OldTimesClan,16,2019-06-01 18:48:01,pt,2019-06-01 23:03:24


In [33]:
query = "SELECT COUNT(DISTINCT(game_id)) FROM game_information;"

pd_sql.read_sql(query, connection)

,count
0,411


In [37]:
query = "SELECT COUNT(DISTINCT(game_name)) FROM game_genres;"

pd_sql.read_sql(query, connection)

,count
0,1001


In [47]:
query = '''SELECT stream_id, user_id, user_name, stream_data.game_id, stream_type, title, viewer_count, started_at, language, time_logged, game_information.game_name, pic_url, game_genres.game_genres 
FROM stream_data 
LEFT JOIN game_information ON stream_data.game_id = game_information.game_id
LEFT JOIN game_genres ON game_information.game_name = game_genres.game_name;'''

data = pd_sql.read_sql(query, connection)

In [48]:
data.head(3)

,stream_id,user_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,RPG
1,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,Adventure Game
2,34366704544,190582624,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,MMORPG


In [50]:
data.to_csv('merged_data.csv')

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339757 entries, 0 to 1339756
Data columns (total 13 columns):
stream_id       1339757 non-null object
user_id         1339757 non-null object
user_name       1339757 non-null object
game_id         1339757 non-null object
stream_type     1339757 non-null object
title           1339757 non-null object
viewer_count    1339757 non-null int64
started_at      1339757 non-null datetime64[ns]
language        1339757 non-null object
time_logged     1339757 non-null datetime64[ns]
game_name       1339691 non-null object
pic_url         1339691 non-null object
game_genres     1338453 non-null object
dtypes: datetime64[ns](2), int64(1), object(10)
memory usage: 132.9+ MB


In [52]:
data.describe()

,viewer_count
count,1.339757e+06
mean,1.412036e+02
std,1.451755e+03
min,0.000000e+00
25%,3.000000e+00
50%,9.000000e+00
75%,3.300000e+01
max,2.435640e+05
